In [1]:
import pandas as pd
import numpy as np

In [10]:
from Bio import SeqIO

In [8]:
try:
    # Attempt to read the CSV file, skipping lines with parsing errors
    lncBase = pd.read_csv('LncBasev2_download.csv', delimiter='\t')
except pd.errors.ParserError as e:
    print(f"ParserError: {e}")

In [9]:
lncBase.head()

,geneId,geneName,mirna,species,cell_line,tissue,category,method,positive_negative,direct_indirect,condition
0,ENSG00000002079,MYH16,hsa-miR-4786-3p,Homo sapiens,293S,Kidney,Embryonic/Fetal,HITS-CLIP,POSITIVE,DIRECT,treatment: emetine
1,ENSG00000067601,PMS2P4,hsa-miR-24-3p,Homo sapiens,NaN,Brain,Normal/Primary,HITS-CLIP,POSITIVE,DIRECT,NaN
2,ENSG00000073905,VDAC1P1,hsa-miR-1179,Homo sapiens,293S,Kidney,Embryonic/Fetal,HITS-CLIP,POSITIVE,DIRECT,no treatment (control)
3,ENSG00000073905,VDAC1P1,hsa-miR-139-5p,Homo sapiens,NaN,Brain,Normal/Primary,HITS-CLIP,POSITIVE,DIRECT,NaN
4,ENSG00000073905,VDAC1P1,hsa-miR-27a-3p,Homo sapiens,NaN,Brain,Normal/Primary,HITS-CLIP,POSITIVE,DIRECT,NaN


In [18]:
# Step 1: Parse the .gtf file
def parse_gtf(gtf_file):
    gtf_data = {}
    with open(gtf_file, 'r') as file:
        for line in file:
            if not line.startswith('#'):
                fields = line.strip().split('\t')
                if fields[2] == "transcript":
                    gene_id = [x for x in fields[8].split(';') if "gene_id" in x][0].split("\"")[1]
                    # Remove version from gene_id
                    gene_id = gene_id.split('.')[0]
                    gtf_data[gene_id] = {
                        'chromosome': fields[0],
                        'start': int(fields[3]),
                        'end': int(fields[4])
                    }
    return gtf_data

In [12]:
gtf_file = "gencode.v29.long_noncoding_RNAs.gtf"
gtf_data = parse_gtf(gtf_file)

In [19]:
# Step 2: Extract sequence using coordinates
def extract_sequence(fasta_file, gtf_data):
    sequences = {}
    for record in SeqIO.parse(fasta_file, "fasta"):
        for gene_id, coords in gtf_data.items():
            if record.id == coords['chromosome']:
                sequences[gene_id] = str(record.seq[coords['start']:coords['end']])
    return sequences


In [20]:
fasta_file = "gencode.v29.transcripts.fa"
sequences = extract_sequence(fasta_file, gtf_data)

In [21]:
# Step 3: Append the sequence to your dataframe
lncBase['sequence'] = lncBase['geneId'].map(sequences)

In [22]:
lncBase.head()

,geneId,geneName,mirna,species,cell_line,tissue,category,method,positive_negative,direct_indirect,condition,sequence
0,ENSG00000002079,MYH16,hsa-miR-4786-3p,Homo sapiens,293S,Kidney,Embryonic/Fetal,HITS-CLIP,POSITIVE,DIRECT,treatment: emetine,NaN
1,ENSG00000067601,PMS2P4,hsa-miR-24-3p,Homo sapiens,NaN,Brain,Normal/Primary,HITS-CLIP,POSITIVE,DIRECT,NaN,NaN
2,ENSG00000073905,VDAC1P1,hsa-miR-1179,Homo sapiens,293S,Kidney,Embryonic/Fetal,HITS-CLIP,POSITIVE,DIRECT,no treatment (control),NaN
3,ENSG00000073905,VDAC1P1,hsa-miR-139-5p,Homo sapiens,NaN,Brain,Normal/Primary,HITS-CLIP,POSITIVE,DIRECT,NaN,NaN
4,ENSG00000073905,VDAC1P1,hsa-miR-27a-3p,Homo sapiens,NaN,Brain,Normal/Primary,HITS-CLIP,POSITIVE,DIRECT,NaN,NaN


In [24]:
missing_ids = lncBase[~lncBase['geneId'].isin(sequences.keys())]['geneId'].unique()
print(f"Missing gene IDs: {len(missing_ids)}")

Missing gene IDs: 8216


In [25]:
print(list(sequences.keys())[:5])

[]


In [23]:
filtered_df = lncBase[lncBase['sequence'].notna()]
print(filtered_df)

Empty DataFrame
Columns: [geneId, geneName, mirna, species, cell_line, tissue, category, method, positive_negative, direct_indirect, condition, sequence]
Index: []
